In [1]:
pip install TPOT


  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11955 sha256=b2b8b9c05e94d8543d891267f5996041707701e18dc74aad7b23bcc9e9e5085e
  Stored in directory: c:\users\hp\appdata\local\pip\cache\wheels\a8\bb\8f\6b9328d23c2dcedbfeb8498b9f650d55d463089e3b8fc0bfb2
Successfully built stopit


In [2]:
import pandas as pd
import numpy as np

In [3]:
from tpot import TPOTRegressor

C:\Users\HP\anaconda3\lib\site-packages\tpot\builtins\__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


In [4]:
X_train = np.load('X_train.npy')
y_train = np.load('y_train.npy')
X_test = np.load('X_test.npy')
y_test = np.load('y_test.npy')

In [ ]:
training_targets = np.array(train_labels).reshape((-1,))
testing_targets = np.array(test_labels).reshape((-1,))

In [9]:
# Create a tpot object with a few parameters
tpot = TPOTRegressor(scoring = 'neg_mean_absolute_error', 
                    max_time_mins = 480, 
                    n_jobs = -1,
                    verbosity = 2,
                    cv = 3)

In [10]:
tpot.fit(X_train, y_train)

Optimization Progress:   0%|          | 0/100 [00:00<?, ?pipeline/s]


5.44 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: SGDRegressor(input_matrix, alpha=0.0, eta0=1.0, fit_intercept=False, l1_ratio=1.0, learning_rate=invscaling, loss=epsilon_insensitive, penalty=elasticnet, power_t=50.0)


TPOTRegressor(max_time_mins=4, n_jobs=-1, scoring='neg_mean_absolute_error',
              verbosity=2)

In [11]:
# Show the final model
print(tpot.fitted_pipeline_)

Pipeline(steps=[('sgdregressor',
                 SGDRegressor(alpha=0.0, eta0=1.0, fit_intercept=False,
                              l1_ratio=1.0, loss='epsilon_insensitive',
                              penalty='elasticnet', power_t=50.0))])


In [ ]:
# Export the pipeline as a python script file
tpot.export('tpot_exported_pipeline.py')

para cuando lo probemos en Colab

In [ ]:
# Import file management
from google.colab import file

# Download the pipeline for local use
files.download('tpot_exported_pipeline.py')

In [12]:
# Esto imprime los piplines pero cuidado, va a haber muchos
# To examine all fitted models
# tpot.evaluated_individuals_

In [ ]:
# Evaluate the final model
print(tpot.score(testing_features, testing_targets))

In [ ]:
# Imports that the final pipeline needs
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LassoLarsCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import Imputer
from tpot.builtins import StackingEstimator

# Preprocessing steps
imputer = Imputer(strategy="median")
imputer.fit(training_features)
training_features = imputer.transform(training_features)
testing_features = imputer.transform(testing_features)

# Final pipeline from TPOT
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=LassoLarsCV(normalize=True)),
    GradientBoostingRegressor(alpha=0.95, learning_rate=0.1, loss="lad", 
                              max_depth=7, max_features=0.75, 
                              min_samples_leaf=3, min_samples_split=18, 
                              n_estimators=100, subsample=0.60)
)

In [ ]:
# Fit on the training data
exported_pipeline.fit(training_features, training_targets)

In [ ]:
# Make predictions on the testing data
predictions = exported_pipeline.predict(testing_features)

print('Mean Absolute Error = %0.4f' % np.mean(abs(predictions - testing_targets)))